<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/knlp_korean_preproc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/e9t/nsmc.git # 한국 영화 평론 데이터 다운

In [0]:
import os  
os.chdir('/content/nsmc') # 파일 다운 위치
!ls # 존재하는 파일 확인
from google.colab import files #파일 업로드를 위한 라이브러리
files.upload() # 파일 업로드 코드

In [0]:
import os
os.chdir('/content/') # 위치 안꼬이게 하려고
!git clone https://github.com/NLP-kr/tensorflow-ml-nlp.git # NLP를 위한 그거
os.chdir('/content/tensorflow-ml-nlp')
!pip install -r requirements.txt # 저자가 제안하는 패키지 목록

In [0]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

os.chdir('/content/nsmc') # 솔직히 큰 의미는 없어보임

In [0]:
DATA_IN_PATH = '/content/nsmc/' # 데이터가 받아진 장소. 여기 있는 파일들을 쓰겠다고 하는 정도로 사용
print("파일크기: ")
for file in os.listdir(DATA_IN_PATH):
  if 'txt'in file:
   print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file) / 1000000, 2)) + 'MB') # 걍 뭐.. 파일 용량

In [0]:
train_data = pd.read_csv(DATA_IN_PATH + 'ratings_train.txt', header = 0, delimiter = '\t', quoting = 3) # 파일 dataframe으로 가져오기
train_data.head() # 데이터 내용 간략 표시

In [0]:
print('전체 학습 데이터의 개수: {}'.format(len(train_data)))

In [0]:
train_length = train_data['document'].astype(str).apply(len) # 각 케이스 별 '한글별 길이' 집계
train_length.head() # 

In [0]:
plt.figure(figsize = (12,5)) # 그래프 그리기 
plt.hist(train_length, bins = 200, alpha = 0.5, color = 'r', label = 'word') # 첫번째는 데이터, 두번쨰는 범위 수(1,2,3,4,5 면 5개, 1~10, 11-20, 이러면 2개 등)
plt.yscale('log', nonposy = 'clip') # 스케일은 로그로 
plt.title('Log-Histogram of length of review', color = 'w')
plt.xlabel('Length of review', color = 'w')
plt.ylabel('Number of review', color = 'w')

In [0]:
train_review = [review for review in train_data['document'] if type(review) is str] # str 타입인 리뷰 데이터만 가져오기

In [0]:
wordcloud = WordCloud(font_path = DATA_IN_PATH + 'NanumGothic.ttf').generate(' '.join(train_review)) # 워드 클라우드 그리기.. 

plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [0]:
fig, axe = plt.subplots(ncols = 1)
fig.set_size_inches(6, 3)
sns.countplot(train_data['label'])

In [0]:
train_word_counts = train_data['document'].astype('str').apply(lambda x: len(x.split(' ')))

plt.figure(figsize = (15, 10))
plt.hist(train_word_counts, bins = 50, facecolor = 'r', label = 'train')
plt.title('Log-Hist', fontsize = 15)
plt.yscale('log', nonposy = 'clip')
plt.legend()
plt.xlabel('Num_word', color = 'w')
plt.ylabel('Num_review', color = 'w')

In [0]:
# 데이터 전처리

import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

DATA_IN_PATH = '/content/nsmc/'

train_data = pd.read_csv(DATA_IN_PATH + 'ratings_train.txt', header = 0, delimiter = '\t', quoting = 3) # 훈련 데이터의 df 화

In [0]:
train_data['document'][0:5] # 아직 정제 안됨

In [0]:
review_text = re.sub("[^가-힣ㄱ-하-ㅣ\\s]","",train_data['document'][4]) # 정제1: 특수문자를 제외한 한글만 정리 - 이 부분은 추후 고민. 순 한글 외 영문도 필요
print(review_text) # 다만 얘는 아직 전체 정제가 아니라 케이스 하나 시범적으로 한거

In [0]:
okt = Okt() # OKT는 형태소 분석에 관한 것,  Komoran, Hannanum, Kkma 이 같은 역할을 수행하나 각자 성능이나 형태소 검출 방식이 다름. 여기선 okt로
review_text = okt.morphs(review_text, stem=True) # 정제 2: 한글만 남은 문장을 통과시킴. 그러면 각 어절 별로 기본형으로 변화시킴
print(review_text)

In [0]:
stop_words = set(['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']) # stop 워드.. 고민이 필요하지만 일단 해당 것들을 제외
clean_review = [token for token in review_text if not token in stop_words] # 
clean_review

In [0]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []): # 전처리를 위한 함수 정의 
#   함수의 인자는 다음과 같다.
#   review: 전처리할 텍스트
#   okt: okt 객체를 반복적으로 생성하지 않고 미리 생성한 후 인자로 받는다. # 전처리 인자를 정해주는 것이겠지
#   remove_stopword: 불용어를 제거할지 여부 선택
#   stop_word: 불용어 사전은 사용자가 직접 입력해야 함.
#   1. 한글 및 공백을 제외한 문자를 모두 제거
  review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]","",review) # DF로 처리되나보다? 아니더라;;
  
#   2. okt 객체를 이용해 형태소 단위로 나눈다.
  word_review = okt.morphs(review_text, stem= True) # okt를 이용하고 위에서 정제된 review_text를 투입 하는데 stem은 단어에서 어간을 추출함
  
  if remove_stopwords: # 위의 스탑워드 제거가 True라면 아까 위에서 한 스탑워드 제거를 실시
    word_review = [token for token in word_review if not token in stop_words]
  
  return word_review # 그래서 케이스별로 어절이 쪼개진 걸 내보내도록 함. 결과로 나오는 단어는 리스트화 된 문자열임

In [0]:
stop_words = set(['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']) # 미리 스탑워드와
okt = Okt() # 형태소 분석기 Okt를 정의
clean_train_review = [] # 그리고 정제된 데이터를 받을 수 있도록 함..

for review in train_data['document']: # 지금 케이스 한 케이스 집어넣고 있는거임;;;
  # 비어있는 데이터에서 멈추지 않도록 문자열인 경우에만 진행
  if type(review) == str: # 들어간 케이스가 문자열이라면
    clean_train_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words = stop_words)) 
    # 위에서 만든 빈 리스트에 데이터를 넣는데, 걔는 preprocessing을 거쳐서 나온거임
  else:
    clean_train_review.append([]) # string이 아니면 비어있는 값 추가, 순서 맞춰야지

clean_train_review[:4] # 그렇게 맞춘 데이터 형태 탐색

In [0]:
# clean_train_review_csv = pd.DataFrame(clean_train_review)
# clean_train_review_csv = clean_train_review_csv.to_csv('/content/nsmc/clean_train_review_csv.csv', encoding='ms949') # 얘는 뭐 한글 쓰기 위함

In [0]:
!ls

In [0]:
# from google.colab import files
# files.download('/content/nsmc/clean_train_review_csv.csv') 

In [0]:
test_data = pd.read_csv(DATA_IN_PATH + 'ratings_test.txt',header = 0, delimiter = '\t', quoting = 3) # 테스트 데이터 임포트

clean_test_review = [] # 역시 빈 그거

for review in test_data['document']: 
  # 빈 데이터에서 멈추지 않도록 문자열인 경우만 진행
  if type(review) == str:
    clean_test_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words = stop_words))
  else:
    clean_test_review.append([]) # string이 아니면 비어있는 값 추가 위와 같은 처리

In [0]:
clean_test_review[:10]

In [0]:
tokenizer = Tokenizer() # Tokenizer는 keras의 클래스
tokenizer.fit_on_texts(clean_train_review) # 단어들에 대해 index까지만 만들어주는 기능으로 tokenizer는 해당 인덱스를 가지게 됨
train_sequences = tokenizer.texts_to_sequences(clean_train_review) # tokenizer가 가진 인덱스에 맞춰서 시퀀스에 숫자를 넣어 줌 (train)
test_sequences = tokenizer.texts_to_sequences(clean_test_review) # 위랑 같지 뭐(test)

word_vocab = tokenizer.word_index # 얘는 tokenizer가 가지고 있는 word_index dict(단어:인덱스)를 복사해주는 것

MAX_SEQUENCE_LENGTH = 8 # 이건 통계적으로 따왔다고 함. 나중에 이걸 20단어 이렇게 가야하려나... 

train_inputs = pad_sequences(train_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post') 
# 패딩함. 8개 단어 자리를 생성하고 리뷰가 8개 단어 이하면 모자란 자리는 0을 넣어준다.얘는 input이고 
train_labels = np.array(train_data['label'])
# 얘는 감정 그거를 만들어 줌 

test_inputs = pad_sequences(test_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post')
test_labels = np.array(test_data['label'])
# 위랑 같음 ㅇㅇ

In [0]:
type(word_vocab)

In [0]:
# 전처림 저장하기
DATA_IN_PATH = '/content/nsmc/'
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'  # npy는 무엇일까...
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs = {} # 사전형 자료 

data_configs['vocab'] = word_vocab # 단어로는 위에서 뽑은 
data_configs['vocab_size'] = len(word_vocab)+1  
#전체 단어 인덱스들이 하나의 vocab 값이고, vocab_size는 전체 수가 된다. 이것의 의미를 알아보는게 필요

import os
if not os.path.exists(DATA_IN_PATH):
  os.makedirs(DTA_IN_PATH)

np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs) # npy 타입으로 input list를 저장
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels) # npy 타입으로 label list를 저장

np.save(open(DATA_IN_PATH + TEST_INPUT_DATA, 'wb'), test_inputs) # 위랑 같음
np.save(open(DATA_IN_PATH + TEST_LABEL_DATA, 'wb'), test_labels)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii = False) # data_configs는 json 문서로 저장.. 왜인지 알아야 겠는데...;

In [0]:
# cnn 방법을 적용할 예정
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split 

In [0]:
DATA_IN_PATH = '/content/nsmc/'
DATA_OUT_PATH = '/content/nsmc/data_out/'
INPUT_TRAIN_DATA_FILE_NAME = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'nsmc_train_label.npy'
DATA_CONFIGS_FILE_NAME = 'data_configs.json'

input_data = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'rb')) # input data 불러오기 
label_data = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'rb')) # label data 불러오기
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS_FILE_NAME, 'r')) # data_Configs.. 인덱스와 인덱스 길이를 가진 애 불러오기

In [0]:
# 옵션값 넣는데..
TEST_SPLIT = 0.1 # 트레이닝 셋을 트레이닝과 테스트 셋으로 나누는 비율
RNG_SEED = 13371447 # 랜덤시드
VOCAB_SIZE = prepro_configs['vocab_size'] # VACAB_SIZE 설정
EMB_SIZE = 128 # embedding ...
BATCH_SIZE = 16 # 배치당 크기
NUM_EPOCHS = 1 # 에폭 수

input_train, input_eval, label_train, label_eval = train_test_split(input_data, label_data, test_size = TEST_SPLIT, random_state = RNG_SEED)
# 데이터 쪼개기

In [0]:
def mapping_fn(X, Y):
  input, label = {'x': X}, Y
  return input, label
  # 매핑 함수.. X, Y를 넣으면 X는 사전형태로 내는데, 'x'가 키로 주어지고 값으로 X가 들어감, Y는 라벨로 리턴됨
  # 'x'와 X를 사전으로 만드는 이유는 들어가는 값은 여러개가 들어가서인데... 

def train_input_fn(): # 얘가 발동하면
  dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train)) # input과 label을 받아서 자를 준비를 한다.일단 매칭함
  dataset = dataset.shuffle(buffer_size = len(input_train)) # 둘이 섞어버린다. 보통 버퍼사이즈는 뭐(배치 사이즈? 데이터 전체?) 보다 더 커야 한다고 한다.
  dataset = dataset.batch(BATCH_SIZE) # 한번에 넣을 배치
  dataset = dataset.map(mapping_fn) # 위의 매핑 함수 투입, input은 x가 되겠지.. 근데 역시 궁금하다.. 한라인 한라인 이러는 걸까... 
  dataset = dataset.repeat(count=NUM_EPOCHS) # 걍 복붙?
  iterator = dataset.make_one_shot_iterator() # 얘는 반복적 작업이 가능하도록 만들어 줌
  
  return iterator.get_next()
  

def eval_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
  dataset = dataset.shuffle(buffer_size = len(input_eval))
  dataset = dataset.batch(16)
  dataset = dataset.map(mapping_fn)
  iterator = dataset.make_one_shot_iterator()

  return iterator.get_next()

In [0]:
train_input_fn()

In [0]:
def model_fn(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN #mode 값에 따라 역할 설정 뭐 도와주나 봄
    EVAL = mode == tf.estimator.ModeKeys.EVAL 
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embedding_layer = tf.keras.layers.Embedding( # embeding layer 생성을 위한 층 생성
                    VOCAB_SIZE,
                    EMB_SIZE)(features['x'])

    dropout_emb = tf.keras.layers.Dropout(rate = 0.2)(embedding_layer) # 거기에 드롭아웃 적용, 여기까지는 모든 모델 적용
    
    conv = tf.keras.layers.Conv1D(  # 1차원 컨볼루션 적용, 컨볼루션 층이 하나인 것이다. conv - pool 페어로 여러 층 쌓을 수 있음
           filters=32,
           kernel_size=3,
           padding='same',
           activation=tf.nn.relu)(dropout_emb)
  
    pool = tf.keras.layers.GlobalMaxPool1D()(conv) # 위에서 커널을 3개로 만들었다면 이제 걔들 중에서 제일 큰 값들 추출(MAXPOOL, 요즘에 더 좋은거 있다던데..)

    hidden = tf.keras.layers.Dense(units=250, activation=tf.nn.relu)(pool) #위에서 Pool로 만들어진 필터?들과 250개 유닛을 가진 히든레이어 하나랑 풀리 커넥트


    dropout_hidden = tf.keras.layers.Dropout(rate=0.2)(hidden, training = TRAIN) 
    # 히든 레이어에 드롭아웃 적용, 여기 트레이닝은 뭔지 알아야겠네
    logits = tf.keras.layers.Dense(units=1)(dropout_hidden) 
    # dropout이 적용되어 훈련이 마친 곳에 1개의 유닛을 가진 레이로 병합(얘가 바로 로짓값 되나?)
    # 기본적으로 keras.layers.Dense의 구성식은 f(Wx+b)의 형태 W와 b는 만들어지고 x는 투입되고 
    # 머신러닝에선 이 값이 구해지고 난 뒤에 activation 단계를 거친다. relu나 시그모이드를 쓰는데
    # 현재 이 식의 다음에는 그 엑티베이션이 없는 것이다. 그러니 얘는 현재 실수 범위 내에서 값을 가진 상태임

    if labels is not None:
        labels = tf.reshape(labels, [-1, 1]) # 라벨 인수가 비어있으면 global의 labels를 가져오고
        
    if TRAIN:
        global_step = tf.train.get_global_step() # train 반복되는거
        loss = tf.losses.sigmoid_cross_entropy(labels, logits) 
        # 이게 로짓의 개념에 맞는 변환, 첫번째 인수 라벨은 정답값.
        # 두번째 인수 logits은 이름만 로짓임, 그 정체는 앞에서 정의한 신경망의 예측 값임, 
        # logits가 0~1로 스케일이 맞춰지지 않았다는데 그럼 범위가 어떻다는 걸까? 실수 범위 내에서 값 가질 수 있음
        # 그럼 label과 logit간 그냥 단순 비교 해도 되는가? -무 +무 다되는데.. 
        # 암튼 뭐 되긴 되겠지, 시간이 엄청 걸리거나 아님 국소최소값에 걸릴 수 있을테지만 일단 안되는 건 아님
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step) 
        # learning rate 0.001의 학습, loss를 최소화 하며 global_step 단계에 따라 업데이트

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss) 
      # 학습모드의 모델 반환, 모드를 받고 loss에 따라 최소화된 가중치들을 내놓는다. 
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits) # 위와 마찬가지로 TRAIN이 만든 모델을 기반으로 예측 비교 점수
        pred = tf.nn.sigmoid(logits) # -무한 +무한의 범위를 가진 logits을 sigmoid 화 하며 0~1의 값을 갖게 함 
        accuracy = tf.metrics.accuracy(labels, tf.round(pred)) 
        # 첫번째 인수는 정답함수, 두번째 함수는 예측값, 정확도
        # tf.metrics.accuracy는 러프하게 총 개수 중에서 맞은게 몇개 나왔는가를 재는 것이라는데...
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        # 위에서 주어진 조건 내에서 가중치 도출
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        ) #에... 이따가...

In [0]:
est = tf.estimator.Estimator(model_fn, model_dir="/content/nsmc/data_out/checkpoint/cnn_model") 
# model_dir은 model_fn으로 만들어진 output(체크포인트와 이벤트 파일)을 저장하는 장소, 
# estimaor는 크게 두 종류로 볼 수 있다. pre-made가 그것들인데 DNNclassifier, DNNregressor 등이 그것이며, 보통은 tf.estimator.DNNclassifier 식으로 쓴다. 
# 그와 반대로 직접 모델을 만드는 custom 모드가 있으며, 이를 실행시키기 위해서는 model_fn을 인자로 넣어줘야 한다.
# model_fn 함수는 아래와 같은 파라미터를 가지며, tf.estimator.EstimatorSpec로 값이 나온다.

  # features: train, evaluate, predict 함수에서 호출될때 input_fn으로 전달되는 첫 번째 값입니다.
  # labels: train, evaluate, predict 함수에서 호출될때 input_fn으로 전달되는 두 번째 값입니다.
  # mode: 학습, 평가 또는 추론여부를 확인 하는 변수로 선택 값입니다.
  # params: Estimator 객체로 부터 전달되는 파라미터 값으로 선택 값입니다.
  # config: configuration 객체로 선택 값입니다.

In [0]:
time_start = datetime.utcnow()
print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 

est.train(train_input_fn)

time_end = datetime.utcnow()
print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))

In [0]:
valid = est.evaluate(eval_input_fn)

In [0]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'

test_input_data = np.load(open(DATA_IN_PATH + INPUT_TEST_DATA, 'rb'))
test_label_data = np.load(open(DATA_IN_PATH + LABEL_TEST_DATA, 'rb'))

In [0]:
def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((test_input_data, test_label_data))
    dataset = dataset.batch(16)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [0]:
predict = est.evaluate(test_input_fn)

In [0]:
predict

In [0]:
!ls

In [0]:
test_input_data = np.load(open(DATA_IN_PATH + TEST_INPUT_DATA, 'rb'))  # 테스트 데이터 설정
ids = np.load(open(DATA_IN_PATH + TEST_ID_DATA, 'rb')) # 위에서 ID 데이터란걸 만들었네... 

predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":test_input_data}, shuffle=False)

predictions = np.array([p['prob'] for p in cnn_est.predict(input_fn=predict_input_fn)])